In [1]:
!pip install evaluate rouge_score bert_score openai

In [2]:
#Accuracy & F1
import evaluate
import numpy as np
import evaluate
import bert_score

# Example data
preds_a = [1, 0, 1, 1, 0]  # Model A predictions
preds_b = [1, 0, 1, 0, 0]  # Model B predictions
labels  = [1, 0, 0, 1, 0]  # Ground truth

accuracy = evaluate.load("accuracy")

print("Model A Accuracy:", accuracy.compute(predictions=preds_a, references=labels)["accuracy"])
print("Model B Accuracy:", accuracy.compute(predictions=preds_b, references=labels)["accuracy"])


Model A Accuracy: 0.8
Model B Accuracy: 0.6


In [3]:
#Accuracy - When full match matters
y_true = ["paris", "tokyo", "madrid"]
y_pred = ["paris", "tokyo", "barcelona"]
accuracy = sum([1 for t, p in zip(y_true, y_pred) if t == p]) / len(y_true)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [4]:
#F1 Score - when partial correctness matters
def f1_score(prediction, ground_truth):
    pred_tokens = prediction.lower().split()
    truth_tokens = ground_truth.lower().split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * (precision * recall) / (precision + recall)

print("F1:", f1_score("The capital is Paris", "Paris is the capital"))


F1: 1.0


In [5]:
#BLEU/ROUGE
import evaluate
# metrics = [ "bleu", "rouge"] as needed
# For summarization:

generated_summaries = [
    "The cat is on the mat"
]

reference_summaries = [
    "The cat is on the mat"]
rouge_metric = evaluate.load("bleu")
result = rouge_metric.compute(predictions=generated_summaries, references=reference_summaries)
print(result)

{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 6, 'reference_length': 6}


In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import math

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model.eval()

text = "The future of AI is"
input_ids = tokenizer.encode(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

print("Perplexity:", perplexity.item())

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Perplexity: 60.44236755371094


In [7]:
from bert_score import score
cands = ["The capital of France is Paris"]
refs = ["Paris is the capital of France"]
P, R, F1 = score(cands, refs, lang="en", verbose=False)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.9314, Recall: 0.9453, F1: 0.9383


In [8]:

import os
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai.api_key)

def gpt_judge(question, response_a, response_b):
    prompt = f"""
You are an expert evaluator.

Question: {question}

Response A: {response_a}
Response B: {response_b}

Which response is more accurate, fluent, and helpful? Reply only with 'A' or 'B'.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

question = "What are the key benefits of using renewable energy sources?"

response_a = "Renewable energy sources like solar and wind reduce carbon emissions, provide sustainable power, and help fight climate change."

response_b = "Using renewable energy is good. It’s better than fossil fuels and it's the future."

result = gpt_judge(question, response_a, response_b)
print("GPT-4 Judge Says:", result)


GPT-4 Judge Says: A
